## Pour éxécuter ce notebook : 

- télécharger les données 3d ici https://we.tl/t-g9Y9v6dROU et les placer dans le répertoire `3d_data/`
- ce code est assez "computationally intensive", mieux vaut une bonne machine pour l'éxecuter
- les données de sortie peuvent être télécharger directement ici : https://we.tl/t-lUM3emszGz

## Ce notebook sert à créer le modèle graphique suivant : 

**Format de l'entrée :** 
- 1ère ligne : $n, m$ deux entiers. $n$ le nombre points, $m$ nombre d'arrêtes.
- $n$ lignes suivantes : matrice $P \in [0, 1]^{n \times 6}$. $P_{ic} =$ probabilité que le points $i$ soit de label $c$ (d'après le classifier).
- $m$ lignes suivantes : arrêtes au format $i, j, d_{ij} \in \{0, ..., n-1\} \times \{0, ..., n-1\} \times \mathbb R_+$ 

On a $n \approx 10^6$ et $m \approx 5 n$

**Format de la sorties :**
- n lignes : $x \in \{1, ..., 6\}^n$ résulats de la minimisation de l'énergie $E(x)$ avec BP ou TRW. 

**Energie $E(x)$:**

$E(x) = \sum_{i=1}^n f_{x_i}(P_{i, x_i}) +  \sum_{(i, j) \in \mathcal E} g(d_{ij}) \mathbb 1_{x_i \neq x_j} $

Avec les $f_c : [0, 1] \rightarrow \mathbb R_-$ décroissantes et $g : \mathbb R_+ \rightarrow \mathbb R_+$ décroissante. Pour commencer on peut prendre : 

$E(x) = \sum_{i=1}^n - \eta_{x_i} P_{i, x_i} +  \sum_{(i, j) \in \mathcal E} \alpha \frac{1}{\beta + d_{ij}} \mathbb 1_{x_i \neq x_j} $ avec $\alpha, \beta > 0$ à régler et $\eta_{c} = \sqrt{\frac{\sum_{i, k} P_{i, k}}{\sum_i P_{i,c}}}$

Cette modélisation vient exprimer le fait suivant : "deux points qui sont proches ont une forte chance de partager le même label". Le but de cette modélisation/minimisation est de trouver des labels $x$ meilleurs que ceux du classifier (qui sont les $(\arg\max_c P_{ic})_i$).

**Score** :

Le score est calculer par rapports aux vrais labels $x^{true} \in \{0, 1, ..., 6\}^n$ (le label $0$ représente les "unclassifed". Les points "unclassifed" n'interviennent pas dans le score.) 

$\text{Score}(x^{true}, x) = \frac 1 6 \sum_{c = 1}^6 \text{IoU}(x^{true}, x, c)$ où $\text{IoU}$ signifie Intersection over Union : $$\text{IoU}(x^{true}, x, c) = \frac{ \{i, x^{true}_i = c\} \cap  \{i, x_i = c\} }{\{i, x^{true}_i = c\} \cup  \{i, x_i = c \text{ et } x^{true}_i \neq 0\}}$$


Le classifier fait entre $0.35$ et $0.5$.

In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import lightgbm as lgb
from multiprocessing import Pool
from pathlib import Path

import pickle
from cloudpoints import load_points
from datasets import CloudDataset, MultiDataset, label_names, df_results
import pandas as pd

# Chargement des données 3d

Il y a 4 jeux de données :
- `paris1`
- `paris2`
- `lille1`
- `lille2`

In [2]:
data_path = Path("3d_data/")

paris, paris_labels = load_points(data_path / "MiniParis1.ply")
lille1, lille1_labels = load_points(data_path / "MiniLille1.ply")
lille2, lille2_labels = load_points(data_path / "MiniLille2.ply")
dijon = load_points(data_path / "MiniDijon9.ply")
paris_wh = paris[:, 1] <= 20
paris1, paris1_label = paris[paris_wh], paris_labels[paris_wh]
paris2, paris2_label = paris[~paris_wh], paris_labels[~paris_wh]

datasets = [
    CloudDataset("paris1", paris1, paris1_label),
    CloudDataset("paris2", paris2, paris2_label),
    CloudDataset("lille1", lille1, lille1_labels),
    CloudDataset("lille2", lille2, lille2_labels)
]
datasets

[CloudDataset<paris1, 1896865 points, labeled>,
 CloudDataset<paris2, 2262453 points, labeled>,
 CloudDataset<lille1, 1901853 points, labeled>,
 CloudDataset<lille2, 2500428 points, labeled>]

# Calculs des voisinages (à plusieurs échelles) de chaque points

In [3]:
for dataset in datasets:
    dataset.compute_neighborhoods([(300, 0.25), (150, 0.15), (50, 0.1)])

# Calculs de features locales : 

In [4]:
def compute_4(an, eigvals, eigs):
    verticality = 2/np.pi * np.arcsin(np.abs(eigs[:, -1, -1]))
    linearity = 1 - eigvals[:, 1] / np.minimum(eigvals[:, 0], 1e-8)
    planarity = (eigvals[:, 1] - eigvals[:, 2]) / np.minimum(eigvals[:, 0], 1e-8)
    sphericity = eigvals[:, 2] / np.minimum(eigvals[:, 0], 1e-8)
    return np.vstack((verticality, linearity, planarity, sphericity)).T

def raw_eigenvalues(an, eigvals, eigs):
    return eigvals

def raw_eigenvector(an, eigvals, eigs):
    return eigs.reshape(-1, 9)

def density(ans, eigvals, eigs):
    return np.array([ns.size for ns in ans], dtype=float).reshape(-1, 1)

feature_functions = [compute_4, raw_eigenvalues, raw_eigenvector, density]

for dataset in datasets:
    dataset.compute_features(feature_functions)

/home/infres/alacote/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/home/infres/alacote/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/home/infres/alacote/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
/home/infres/alacote/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


# Entrainement d'un classifier et résultats sur `paris2`

Le classifier est entrainé sur `paris1`, `lille1`, `lille2` et testé sur `paris2` :

In [5]:
test_idx = 1
train_multidataset = MultiDataset(datasets, test_idx)
X_train, X_test, y_train, labels_test = train_multidataset.train_test_split()

array([199742, 790686, 501958,  14467,  10567,  37790, 707243])

In [14]:
freq = np.bincount(labels_test) 
freq = freq / freq.sum()
freq

array([0.02194104, 0.42954246, 0.40030547, 0.0052367 , 0.        ,
       0.03844622, 0.1045281 ])

In [7]:
num_round = 50
param = {'num_leaves': 31, 'max_depth': -1, 'objective': 'multiclass', 'num_class': 6, 'max_bin': 30}

train_data = lgb.Dataset(X_train, label=y_train)
bst = lgb.train(param, train_data, num_round)
y_pred = np.concatenate((np.zeros((X_test.shape[0], 1)), bst.predict(X_test)), axis=1) 
labels_pred = np.argmax(y_pred[:, 1:], axis=1) + 1

res = df_results(labels_test, labels_pred)
print("Score:", res["IoU"].mean())
res

Score: 0.4736162489171199


,precision,recall,IoU
1 - Ground,0.948693,0.952100,0.917495
2 - Building,0.634104,0.859002,0.719627
3 - Poles,0.596179,0.220018,0.195877
4 - Pedestrians,0.333260,0.144317,0.118714
5 - Cars,0.468369,0.088357,0.084589
6 - Vegetation,0.835188,0.911537,0.805395


# Création et sauvegarde de l'entrée décrite ci-dessus pour chaque dataset : 

In [10]:
grm_path = Path("GRM_data")
if(not grm_path.exists()):
    grm_path.mkdir()

for dataset_idx in [0, 1, 2, 3]:
    train_multidataset = MultiDataset(datasets, dataset_idx)
    X_train, X_test, y_train, labels_test = train_multidataset.train_test_split()
    train_data = lgb.Dataset(X_train, label=y_train)
    bst = lgb.train(param, train_data, num_round)
    y_pred = np.concatenate((np.zeros((X_test.shape[0], 1)), bst.predict(X_test)), axis=1) 
    labels_pred = np.argmax(y_pred[:, 1:], axis=1) + 1
    
    res = df_results(labels_test, labels_pred)
    print("avg IoU:", res["IoU"].mean())
    worst_class = res["IoU"].idxmin()
    print("Worst class :", worst_class, "| IoU:", res.loc[worst_class, "IoU"])
    
    datasets[dataset_idx].write_GRM(y_pred[:, 1:], grm_path, grm_k=15, grm_radius=0.1, grm_min_k=3)

avg IoU: 0.36206471884456626
Worst class : 5 - Cars | IoU: 0.014310664626798987
avg IoU: 0.4826614824699731
Worst class : 4 - Pedestrians | IoU: 0.10407169898786989
avg IoU: 0.4407801774965228
Worst class : 3 - Poles | IoU: 0.03163660598991961
avg IoU: 0.5040229855883136
Worst class : 5 - Cars | IoU: 0.11176871622088945
